# 第5章 ハッシュ関数とブロックチェーン
- 学生番号：
- 氏名：

## ハッシュ関数 SHA256 によるハッシュ値の計算

## 実習5.1
+ 以下のコードを実行しなさい.
+ myStr2 として，"私の成績はAです．" とすることでハッシュ値を計算し，比較しなさい
    + セルまたは文字列のコピーを利用すると良いだろう

In [2]:
# SHA256 によるハッシュ値の計算，比較
from hashlib import sha256 # hashを扱うライブラリhashlibを用いる

myStr1 = "私の成績はFです." #ハッシュ値を求める文字列


print(myStr1.encode("UTF-8")) # UTF-8 でエンコードしたものの表示 (この形を使う)
print(myStr1.encode("UTF-8").hex()) # それを16進表示したもの (参考)
print(int(myStr1.encode("UTF-8").hex(),16)) # それを10進表示したもの (参考)

myHash1 = sha256(myStr1.encode("UTF-8")) #sha256でハッシュを求める
myHash1_hex=myHash1.hexdigest() # 16進表示
myHash1_dec=int(myHash1_hex,16) # 10進の整数に変換

print(myHash1_hex) # それぞれ表示
print(myHash1_dec)

b'\xe7\xa7\x81\xe3\x81\xae\xe6\x88\x90\xe7\xb8\xbe\xe3\x81\xafF\xe3\x81\xa7\xe3\x81\x99.'
e7a781e381aee68890e7b8bee381af46e381a7e381992e
22188076321794593932430065946794283322252641200490256686
7c136d5c3ec36338bd15588dc0a13b9ffd7055ab2ebed67dd69ec2389ef3fa9a
56121118095832454883506314253225243795979571668490411751083301587444047280794


## 実習5.2
+ 以下を実行しなさい
+ 条件として，ハッシュ値の最初に0が4つ並ぶものを求めなさい
    + 余裕があれば，条件を変えて，合致するものの頻度についても考えると良いだろう．

In [6]:
from hashlib import sha256

myStr = "ハッシュ値チェック" # ハッシュ値を求める文字列
print(myStr)
myHash = sha256(myStr.encode("UTF-8")) #sha256 でハッシュ値を求める
print(myHash.hexdigest()) # 16進法表示で表示
condStr="000" # 最初がこうなるハッシュを探す
searchLimit=10000 # 試行回数の上限
i=0
checkHash=sha256((myStr+str(i)).encode("UTF-8")).hexdigest()
while (checkHash[:len(condStr)]!=condStr) and (i<searchLimit):
    i+=1
    checkHash=sha256((myStr+str(i)).encode("UTF-8")).hexdigest()
print(myStr+str(i))
print(sha256((myStr+str(i)).encode("UTF-8")).hexdigest())

ハッシュ値チェック
47fcab849ae117a76cd192e6ae09c181d4e17dca578abe971d4b59fc5a30c64c
ハッシュ値チェック4648
00029fc741426258f4493620709b7223cf3b07f6a922b546a5f37b9a15d16d87


## (参考) nonce を用いたブロックチェーンの実装

In [7]:
from hashlib import sha256
from random import random

class Block: # ブロックをクラスとして定義する
    def __init__(self, index, timestamp, data, previous_hash,value): # 属性(Block が持つ要素)の設置
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.value= value
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
        self.nonce = self.proof_of_work()
    def hash_block(self): # ハッシュ値を求めるメソッドの定義
        sha = sha256()
        sha.update((str(self.index) + str(self.timestamp) + 
                    str(self.data) + str(self.value)+ str(self.previous_hash)).encode('utf-8'))
        return sha.hexdigest()
    def proof_of_work(self):
        """
        シンプルなプルーフ・オブ・ワークのアルゴリズム:
         - hash(pp') の最初のnum個が0となるような p' を探す
         - p は1つ前のブロックのプルーフ、 p' は新しいブロックのプルーフ
        :param last_proof: <int>
        :return: <int>
        """
        num=3
        proof = 0
        while self.valid_proof(proof,num) is False:
            proof += 1
        return proof
    def valid_proof(self,proof,num):
        """
        プルーフが正しいかを確認する: hash(last_proof, proof)の最初のnum個が0となっているか？
        :param last_proof: <int> 前のプルーフ
        :param proof: <int> 現在のプルーフ
        :return: <bool> 正しければ true 、そうでなれけば false
        """
        guess = f'{self.hash}{proof}'.encode('UTF-8')
        guess_hash = sha256(guess).hexdigest()
        return guess_hash[:num] == "0"*num

from datetime import datetime

def create_genesis_block(): # 最初のブロックの定義
    # Manually construct a block with
    # index zero and arbitrary previous hash
    return Block(0, datetime.now(), "Genesis Block", "", 0)

def next_block(last_block,value): # 次のブロックを設定
    this_index = last_block.index + 1
    this_timestamp = datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_hash = last_block.hash
    this_value = value
    return Block(this_index, this_timestamp, this_data, this_hash, this_value)

# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]
previous_block = blockchain[0]
print("First Block")
print("Hash: {}\nNonce: {}\n".format(blockchain[0].hash,blockchain[0].nonce))

# How many blocks should we add to the chain
# after the genesis block
num_of_blocks_to_add = 3
valueList=[int(10000*random()) for i in range(num_of_blocks_to_add)]



# Add blocks to the chain
for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block,valueList[i])
    blockchain.append(block_to_add)
    previous_block = block_to_add

# Tell everyone about it!
    print("Block #{} has been added to the blockchain!".format(block_to_add.index))
    print("Hash: {}\nNonce: {}\n".format(block_to_add.hash,block_to_add.nonce))

for i in range(0, num_of_blocks_to_add+1):
    iblock=blockchain[i]
    itime=iblock.timestamp.strftime("%H:%M:%S")
    iprevhash=iblock.previous_hash
    ihash=iblock.hash
    idata=iblock.data
    ivalue=iblock.value
    inonce=iblock.nonce
    icheck=sha256(f'{ihash}{inonce}'.encode('UTF-8')).hexdigest()
    print("{}-th block\ndate:{}\nPrev_hash:{}\nThis_hash:{}\nData:{}\nValue:{}\nCheck:{}\n\n".format(iblock.index,itime,iprevhash,ihash,idata,ivalue,icheck))

First Block
Hash: a0c145ad548b14fe505452b8d288d1875c9902041f2e622e89c14634c990e3b6
Nonce: 4534

Block #1 has been added to the blockchain!
Hash: fc027b5f3132820aefa9f0dd8e9d5a2ac49b420f257e29813194e4c55a35eb30
Nonce: 6824

Block #2 has been added to the blockchain!
Hash: 001d072b10c1b2eb0b758c7955a05e2b641ae6d78a5fbafad87461fbd7ce4953
Nonce: 8236

Block #3 has been added to the blockchain!
Hash: c2244f5d57b4e202540d2bb55c0f2c2c9f9c24f0f20f5c4d4d55a6522dd0172c
Nonce: 3

0-th block
date:17:04:55
Prev_hash:
This_hash:a0c145ad548b14fe505452b8d288d1875c9902041f2e622e89c14634c990e3b6
Data:Genesis Block
Value:0
Check:00038768b5a2f610ec7723d31ddca7ecb33d235876acb9322fdd7cfc25d790f6


1-th block
date:17:04:55
Prev_hash:a0c145ad548b14fe505452b8d288d1875c9902041f2e622e89c14634c990e3b6
This_hash:fc027b5f3132820aefa9f0dd8e9d5a2ac49b420f257e29813194e4c55a35eb30
Data:Hey! I'm block 1
Value:4282
Check:00065ba94f8bca1e82c169234c755cc8556aaaae9a96c1e548d0a1f98bf30067


2-th block
date:17:04:55
Prev_hash

## 課題については以下で行うこと